### ENCCS - Python for High Performance Data Analytics - Exercises

#### Exercise 1:

This exercise is derived from Xarray Tutorials, which is distributed under an Apache-2.0 License.
First create an Xarray dataset:

In [1]:
import numpy as np
import xarray as xr

ds1 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(4, 2)),
        "b": (("z", "x"), np.random.randn(6, 4)),
    },
    coords={
        "x": np.arange(4),
        "y": np.arange(-2, 0),
        "z": np.arange(-3, 3),
    },
)
ds2 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(7, 3)),
        "b": (("z", "x"), np.random.randn(2, 7)),
    },
    coords={
        "x": np.arange(6, 13),
        "y": np.arange(3),
        "z": np.arange(3, 5),
    },
)

Then write the datasets to disk using to_netcdf() method:

In [2]:
ds1.to_netcdf("ds1.nc")
ds2.to_netcdf("ds2.nc")

You can read an individual file from disk by using open_dataset() method, or using the load_dataset() method:

In [3]:
ds3 = xr.open_dataset("ds1.nc")
ds4 = xr.load_dataset('ds1.nc')

Tasks:

* Explore the hierarchical structure of the ds1 and ds2 datasets in a Jupyter notebook by typing the variable names in a code cell and execute. Click the disk-looking objects on the right to expand the fields.
* Explore ds3 and ds4 datasets, and compare them with ds1. What are the differences?

In [4]:
# Printing:
ds1

<xarray.Dataset>
Dimensions:  (x: 4, y: 2, z: 6)
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1
  * z        (z) int64 -3 -2 -1 0 1 2
Data variables:
    a        (x, y) float64 0.3278 -1.145 -0.2592 ... 0.3374 -0.8789 0.7119
    b        (z, x) float64 0.2958 0.8275 1.199 -0.7458 ... 0.7284 2.074 0.249

In [5]:
ds2

<xarray.Dataset>
Dimensions:  (x: 7, y: 3, z: 2)
Coordinates:
  * x        (x) int64 6 7 8 9 10 11 12
  * y        (y) int64 0 1 2
  * z        (z) int64 3 4
Data variables:
    a        (x, y) float64 0.9385 0.9135 -1.199 ... 0.3392 -1.778 -0.8377
    b        (z, x) float64 0.7462 -0.9692 0.5921 ... 0.8423 -2.473 -0.1652

In [7]:
# Compare ds3 and ds4 with ds1
ds3


<xarray.Dataset>
Dimensions:  (x: 4, y: 2, z: 6)
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1
  * z        (z) int64 -3 -2 -1 0 1 2
Data variables:
    a        (x, y) float64 ...
    b        (z, x) float64 ...

In [8]:
ds4

<xarray.Dataset>
Dimensions:  (x: 4, y: 2, z: 6)
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1
  * z        (z) int64 -3 -2 -1 0 1 2
Data variables:
    a        (x, y) float64 0.3278 -1.145 -0.2592 ... 0.3374 -0.8789 0.7119
    b        (z, x) float64 0.2958 0.8275 1.199 -0.7458 ... 0.7284 2.074 0.249